In [1]:
import io
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from dateutil import parser
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from lightweight_charts import Chart
# Network configuration
network = 'ethereum'  # @param {type:"string"}
network_ids = {
    'arbitrum': '/42161',
    'ethereum': '/1',
    'mantle': '/5000'
}

# Initialize session with retry mechanism
session = requests.session()
retry = Retry(total=3, backoff_factor=1)
session.mount('http://', HTTPAdapter(max_retries=retry))
session.mount('https://', HTTPAdapter(max_retries=retry))

# Retrieve network ID
network_id = network_ids.get(network.lower())
if network_id is None:
    raise ValueError("Unsupported network type")

# Construct the request URL and headers
url = f'https://api-v2.pendle.finance/core/v1{network_id}/assets/all'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

# Send request and parse response
response = session.get(url, headers=headers)
data = response.json()

# Function to find valid assets based on given parameters
def find_valid_assets(data, base_type, expiry_key, address):
    """
    Find valid assets matching the specified criteria.

    :param data: The asset data returned by the API
    :param base_type: The type of asset to filter by (e.g., 'YT')
    :param expiry_key: The key indicating the expiry date of the asset
    :param address: The contract address to filter by
    :return: A list of valid assets that meet the criteria
    """
    current_time = datetime.utcnow().replace(tzinfo=timezone.utc)

    def parse_to_utc(date_str):
        dt = parser.parse(date_str)
        return dt.astimezone(timezone.utc)

    def format_expiry(date_str):
        dt = parse_to_utc(date_str)
        return dt.strftime('%Y-%m-%d %H:%M:%S')

    valid_assets = [
        {**item, expiry_key: format_expiry(item[expiry_key])} for item in data
        if item.get('baseType') == base_type and
           item.get('address') == address and
           expiry_key in item
    ]

    return valid_assets

# Form parameter configuration
market_contract = "0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08"  # @param {type:"string"}
yt_contract = '0xbe05538f48d76504953c5d1068898c6642937427'  # @param {type:"string"}
start_time = "2023-01-01 00:00:00"
underlying_amount = 1  # @param {type:"number"}
points_per_hour_per_underlying = 1  # @param {type:"number"}
pendle_yt_multiplier = 1  # @param {type:"number"}
dark_mode = True  # @param {type:"boolean"}

# Handle time and chart mode
datetime_obj = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
start_time = datetime_obj.replace(tzinfo=timezone.utc).isoformat(timespec='milliseconds').replace('+00:00', 'Z')
points = points_per_hour_per_underlying

mode = 'plotly_dark' if dark_mode else 'plotly_white'

# Find valid YT assets
valid_assets = find_valid_assets(data, 'YT', 'expiry', yt_contract)
if not valid_assets:
    raise ValueError("No valid assets found with the given parameters")

symbol = valid_assets[0]['symbol']
maturity = valid_assets[0]['expiry']


C:\Users\DC\AppData\Local\Temp\ipykernel_25216\3932284330.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow().replace(tzinfo=timezone.utc)


In [2]:
url_ohlcv_yteth = f'https://api-v2.pendle.finance/core/v3{network_id}/prices/{yt_contract}/ohlcv'

headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
        }

params = {
            "time_frame": "hour",
            "timestamp_start": start_time,
            "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')
        }
response = session.get(url_ohlcv_yteth, headers=headers, params=params)

results = response.json().get('results', [])
data = [
        [datetime.fromisoformat(item['time'].rstrip('Z')), item['open'], item['high'], item['low'], item['close'], item.get('volume', 0)]
        for item in results
        ]

C:\Users\DC\AppData\Local\Temp\ipykernel_25216\4080272874.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')


In [37]:
dfYT = pd.DataFrame(data, columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
dfPT = pd.DataFrame({
    'Time': dfYT['Time'],
    'Open': 1 - dfYT['Open'],
    'High': 1 - dfYT['Low'],  # Note: High becomes Low when inverted
    'Low': 1 - dfYT['High'],  # Note: Low becomes High when inverted
    'Close': 1 - dfYT['Close'],
    'Volume': dfYT['Volume']  # Volume remains the same
})


In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def create_chart(df, title):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('', 'Volume'), 
                        row_heights=[0.7, 0.3])

    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='OHLC'
    ), row=1, col=1)

    colors = ['red' if row['Open'] > row['Close'] else 'green' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(
        x=df.index, 
        y=df['Volume'],
        marker_color=colors,
        name='Volume'
    ), row=2, col=1)

    fig.update_layout(
        title=title,
        height=800,
        width=1200,
        showlegend=False,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='black'),
        xaxis=dict(
            title='Date',
            gridcolor='#333333',
            rangeslider=dict(visible=False)
        ),
        xaxis2=dict(
            title='Date',
            gridcolor='#333333'
        )
    )

    fig.update_yaxes(title_text='Price', row=1, col=1, gridcolor='#333333')
    fig.update_yaxes(title_text='Volume', row=2, col=1, gridcolor='#333333')

    return fig

# Ensure the DataFrame index is datetime
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Create 1-hour timeframe chart
fig_1h = create_chart(dfPT, 'Trading Chart (1-Hour Timeframe)')

# Resample to 1-day timeframe
df_1d = dfPT.resample('1D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

# Create 1-day timeframe chart
fig_1d = create_chart(df_1d, 'Trading Chart (1-Day Timeframe)')

# Show both charts
fig_1h.show()
fig_1d.show()

# Optionally, save the charts to HTML files
fig_1h.write_html("trading_chart_1h.html")
fig_1d.write_html("trading_chart_1d.html")
print("Charts saved as 'trading_chart_1h.html' and 'trading_chart_1d.html'.")

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [40]:
df_1d = dfPT.resample('1D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [41]:
dfPT

,Open,High,Low,Close,Volume
0,0.913406,0.913406,0.913406,0.913406,0.000000
1,0.913518,0.913518,0.913471,0.913486,0.000000
2,0.913487,0.913515,0.913487,0.913515,0.000000
3,0.913532,0.913550,0.913532,0.913537,0.000000
4,0.913532,0.913563,0.913532,0.913563,0.000000
...,...,...,...,...,...
1325,0.970073,0.970084,0.970073,0.970084,7624.469636
1326,0.970085,0.970088,0.970064,0.970073,0.000000
1327,0.970074,0.970085,0.970074,0.970085,0.000000
1328,0.970086,0.970438,0.970086,0.970438,292372.936084
